<a href="https://colab.research.google.com/github/cs19b028iittp/islcourse/blob/main/ISL_ASN_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage
from PIL import Image
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_score

In [51]:
transform_tensor_to_pil = ToPILImage()
transform_pil_to_tensor = ToTensor()

In [52]:
def load_data():
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )
    return training_data, test_data

In [53]:
training_data, test_data = load_data()

print (training_data[0][0].shape)

torch.Size([1, 28, 28])


In [54]:
def create_dataloaders(training_data, test_data, batch_size=64):
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    for X, y in test_dataloader:
        print(f"Shape of X [N, C, H, W]: {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}")
        break
        
    return train_dataloader, test_dataloader

In [55]:
print (len(set([y for x,y in training_data])))

10


In [56]:
train_loader, test_loader = create_dataloaders(training_data, test_data, batch_size = 32)

Shape of X [N, C, H, W]: torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]) torch.int64


In [57]:
class cs19b028(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(1, 16, 28)
        self.m = nn.Softmax(dim =1)
        self.fc1 = nn.Linear(28*28*1, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.m(x)
        return x

In [58]:
y = (len(set([y for x,y in training_data])))
model = cs19b028()

In [59]:
def train_network(train_loader, optimizer,criteria, e):
  for epoch in range(e):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        tmp = torch.nn.functional.one_hot(labels, num_classes= 10)
        loss = criteria(outputs, tmp)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

  print('Finished Training')

In [60]:
#cross entropy
def loss_fun(y_pred, y_ground):
  v = -(y_ground * torch.log(y_pred + 0.0001))
  v = torch.sum(v)
  return v

In [61]:
x,y = training_data[0]
model = cs19b028()
y_pred = model(x)
print(y_pred.shape)
print(y_pred)
print(torch.sum(y_pred))
# loss_fun(10,y_pred)

torch.Size([1, 10])
tensor([[0.0968, 0.1240, 0.1260, 0.1071, 0.0757, 0.0799, 0.0894, 0.0989, 0.1076,
         0.0946]], grad_fn=<SoftmaxBackward0>)
tensor(1., grad_fn=<SumBackward0>)


In [62]:
y_ground = y
loss_val = loss_fun(y_pred, y_ground)
print(loss_val)

y01h = torch.nn.functional.one_hot(torch.tensor(y), num_classes= 10)

# loss_fun(10,y01h)

tensor(208.2729, grad_fn=<SumBackward0>)


In [63]:
loss_fun(y_pred,y01h)

tensor(2.3573, grad_fn=<SumBackward0>)

In [64]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [65]:
train_network(train_loader,optimizer,loss_fun,10)

Finished Training


In [66]:
# test(y01h, model, loss_fun)

In [67]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [68]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
from sklearn.metrics import precision_recall_fscore_support


In [69]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
from torchmetrics import Precision, Recall, F1Score, Accuracy

In [71]:
from torchmetrics.classification import accuracy
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            #X, y = X.to(device), y.to(device)
            tmp = torch.nn.functional.one_hot(y, num_classes= 10)
            pred = model(X)
            test_loss += loss_fn(pred, tmp).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    #precision_recall_fscore_support(y_ground, y_pred, average='macro')
    accuracy1 = Accuracy()
    print('Accuracy :', accuracy1(pred,y))
    precision = Precision(average = 'macro', num_classes = 10)
    print('precision :', precision(pred,y))

    recall = Recall(average = 'macro', num_classes = 10)
    print('recall :', recall(pred,y))
    f1_score = F1Score(average = 'macro', num_classes = 10)
    print('f1_score :', f1_score(pred,y))
    return accuracy1,precision, recall, f1_score

In [72]:
test(test_loader, model, loss_fun)

Test Error: 
 Accuracy: 87.0%, Avg loss: 12.199801 

Accuracy : tensor(1.)
precision : tensor(1.)
recall : tensor(1.)
f1_score : tensor(1.)


(Accuracy(), Precision(), Recall(), F1Score())

In [73]:
#write the get model
def get_model(train_loader,e = 10):
	model = cs19b028()
	optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
	criteria = loss_fun
	train_network(train_loader, optimizer,criteria,e)
	return model
